In [1]:
# Loading the dataset 
import pandas as pd

brick_index_value = pd.read_csv('./bricks_index_values.csv')
brick_rp_distances = pd.read_csv('./brick_rp_distances.csv')
#distances= pd.read_csv('./distances.xlsx')
pfizer =pd.read_csv('./Pfitzer10-100.csv', sep=';')

In [2]:
import gurobipy
from gurobipy import Model, GRB, quicksum
bricks = list(range(1, 23)) 
reps = list(range(1, 5))  

index_values = {
    1: 0.1609, 2: 0.1164, 3: 0.1026, 4: 0.1516, 5: 0.0939, 6: 0.1320, 7: 0.0687,
    8: 0.0930, 9: 0.2116, 10: 0.2529, 11: 0.0868, 12: 0.0828, 13: 0.0975, 14: 0.8177,
    15: 0.4115, 16: 0.3795, 17: 0.0710, 18: 0.0427, 19: 0.1043, 20: 0.0997,
    21: 0.1698, 22: 0.2531
}

distances = {
    (1, 1): 16.16, (1, 2): 24.09, (1, 3): 24.29, (1, 4): 21.12,
    (2, 1): 19.00, (2, 2): 26.47, (2, 3): 27.22, (2, 4): 17.33,
    (3, 1): 25.29, (3, 2): 32.50, (3, 3): 33.41, (3, 4): 12.25,
    (4, 1): 0.00, (4, 2): 7.94, (4, 3): 8.29, (4, 4): 36.12,
    (5, 1): 3.07, (5, 2): 6.45, (5, 3): 7.55, (5, 4): 37.37,
    (6, 1): 1.22, (6, 2): 7.51, (6, 3): 8.17, (6, 4): 36.29,
    (7, 1): 2.80, (7, 2): 10.30, (7, 3): 10.92, (7, 4): 33.50,
    (8, 1): 2.87, (8, 2): 5.07, (8, 3): 5.65, (8, 4): 38.80,
    (9, 1): 3.80, (9, 2): 8.02, (9, 3): 7.37, (9, 4): 38.16,
    (10, 1): 12.35, (10, 2): 4.51, (10, 3): 4.37, (10, 4): 48.27,
    (11, 1): 11.11, (11, 2): 3.51, (11, 3): 2.97, (11, 4): 47.14,
    (12, 1): 21.99, (12, 2): 22.01, (12, 3): 24.10, (12, 4): 39.86,
    (13, 1): 8.82, (13, 2): 3.28, (13, 3): 5.38, (13, 4): 43.31,
    (14, 1): 7.94, (14, 2): 0.00, (14, 3): 2.10, (14, 4): 43.75,
    (15, 1): 9.34, (15, 2): 2.32, (15, 3): 1.11, (15, 4): 45.43,
    (16, 1): 8.29, (16, 2): 2.10, (16, 3): 0.00, (16, 4): 44.40,
    (17, 1): 7.28, (17, 2): 2.40, (17, 3): 1.08, (17, 4): 43.40,
    (18, 1): 7.55, (18, 2): 0.77, (18, 3): 1.54, (18, 4): 43.52,
    (19, 1): 11.13, (19, 2): 18.41, (19, 3): 19.24, (19, 4): 25.40,
    (20, 1): 17.49, (20, 2): 23.44, (20, 3): 24.76, (20, 4): 23.21,
    (21, 1): 11.03, (21, 2): 18.94, (21, 3): 19.26, (21, 4): 25.43,
    (22, 1): 36.12, (22, 2): 43.75, (22, 3): 44.40, (22, 4): 0.00
}

L = 0.8
U = 1.2

In [3]:
# Modèle pour minimiser la distance
model_distance = Model("Minimize_Distance")

# Variables de décision
x = model_distance.addVars(bricks, reps, vtype=GRB.BINARY, name="x")

# Fonction objectif : minimiser la distance totale
model_distance.setObjective(
    quicksum(distances[b, r] * x[b, r] for b in bricks for r in reps), GRB.MINIMIZE
)

model_distance.addConstrs(
    (quicksum(x[b, r] for r in reps) == 1 for b in bricks), name="AssignEachBrick"
)

model_distance.addConstrs(
    (quicksum(index_values[b] * x[b, r] for b in bricks) >= L for r in reps), name="MinWorkload"
)
model_distance.addConstrs(
    (quicksum(index_values[b] * x[b, r] for b in bricks) <= U for r in reps), name="MaxWorkload"
)

model_distance.optimize()

if model_distance.Status == GRB.OPTIMAL:
    print("\nDistance totale minimale :", model_distance.ObjVal)
    for b in bricks:
        for r in reps:
            if x[b, r].X > 0.5:
                print(f"Brick {b} -> Représentant {r}")

Set parameter Username
Set parameter LicenseID to value 2600216
Academic license - for non-commercial use only - expires 2025-12-17
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (mac64[arm] - Darwin 23.6.0 23G80)

CPU model: Apple M3
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 30 rows, 88 columns and 264 nonzeros
Model fingerprint: 0x4ea169f8
Variable types: 0 continuous, 88 integer (88 binary)
Coefficient statistics:
  Matrix range     [4e-02, 1e+00]
  Objective range  [8e-01, 5e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [8e-01, 1e+00]
Found heuristic solution: objective 353.0700000
Presolve time: 0.00s
Presolved: 30 rows, 88 columns, 264 nonzeros
Variable types: 0 continuous, 88 integer (88 binary)

Root relaxation: objective 1.488198e+02, 25 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd

In [4]:
# Hypothèse : Les affectations initiales sont les représentants par défaut (exemple ici avec rp1)
initial_assignment = {b: 1 for b in bricks}

# Modèle pour minimiser la perturbation
model_disruption = Model("Minimize_Disruption")

# Variables de décision
x = model_disruption.addVars(bricks, reps, vtype=GRB.BINARY, name="x")
y = model_disruption.addVars(bricks, reps, vtype=GRB.BINARY, name="y")

# Fonction objectif : minimiser la perturbation pondérée par l'index
model_disruption.setObjective(
    quicksum(index_values[b] * y[b, r] for b in bricks for r in reps), GRB.MINIMIZE
)

# Contraintes
# 1. Chaque brick doit être attribué à un seul représentant
model_disruption.addConstrs(
    (quicksum(x[b, r] for r in reps) == 1 for b in bricks), name="AssignEachBrick"
)

# 2. La charge de travail doit être équilibrée
model_disruption.addConstrs(
    (quicksum(index_values[b] * x[b, r] for b in bricks) >= L for r in reps), name="MinWorkload"
)
model_disruption.addConstrs(
    (quicksum(index_values[b] * x[b, r] for b in bricks) <= U for r in reps), name="MaxWorkload"
)

# 3. Mesurer la perturbation
model_disruption.addConstrs(
    (y[b, r] >= x[b, r] - (1 if initial_assignment[b] == r else 0)
     for b in bricks for r in reps), name="Disruption"
)

# Résoudre le modèle
model_disruption.optimize()

# Afficher les résultats
if model_disruption.Status == GRB.OPTIMAL:
    print("\nPerturbation totale minimale :", model_disruption.ObjVal)
    for b in bricks:
        for r in reps:
            if x[b, r].X > 0.5:
                print(f"Brick {b} -> Représentant {r}")

Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (mac64[arm] - Darwin 23.6.0 23G80)

CPU model: Apple M3
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 118 rows, 176 columns and 440 nonzeros
Model fingerprint: 0x2bbf88e1
Variable types: 0 continuous, 176 integer (176 binary)
Coefficient statistics:
  Matrix range     [4e-02, 1e+00]
  Objective range  [4e-02, 8e-01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [8e-01, 1e+00]
Found heuristic solution: objective 2.8883000
Presolve removed 88 rows and 88 columns
Presolve time: 0.00s
Presolved: 30 rows, 88 columns, 264 nonzeros
Variable types: 0 continuous, 88 integer (88 binary)

Root relaxation: objective 2.800000e+00, 41 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    2.80000    0    6    2.88830    2.80000  3.06%     